# PyTorch 基础 :数据的加载和预处理

PyTorch通过**torch.utils.data**对常用数据加载进行封装，可以容易地实现多线程**数据预读和批量加载**。

torchvision已经预先实现常用图像数据集，包括CIFAR-10，ImageNet、COCO、MNIST、LSUN等，
可通过torchvision.datasets方便调用

In [1]:
# 首先要引入相关的包
import torch

#打印一下版本
torch.__version__

'1.6.0'

## Dataset

Dataset是一个抽象类，为了能够方便读取，需要将要用的数据包装为Dataset类。

自定义的Dataset需要继承它并且实现两个成员方法：
1. `__getitem__()` 定义用索引(`0` 到 `len(self)`)获取一条数据或一个样本
2. `__len__()` 返回数据集的总长度

用kaggle上的一个竞赛[bluebook for bulldozers](https://www.kaggle.com/c/bluebook-for-bulldozers/data)自定义一个数据集，为了方便介绍，使用数据字典来做说明（因为条数少）

In [3]:
# 引用
from torch.utils.data import Dataset
import pandas as pd


# 定义一个数据集
class BulldozerDataset(Dataset):
    """ 
    数据集演示 
    """
    def __init__(self, csv_file):
        """
        实现初始化方法，初始化时将数据读载入
        """
        self.df = pd.read_csv(csv_file)

    def __len__(self):
        '''
        返回df的长度
        '''
        return len(self.df)

    def __getitem__(self, idx):
        '''
        根据 idx 返回一行数据
        '''
        return self.df.iloc[idx].SalePrice

至此，数据集已经定义完成，可以实例化一个对象访问

In [4]:
ds_demo = BulldozerDataset('median_benchmark.csv')

可以直接使用如下命令查看数据集数据

In [5]:
# 实现了 __len__ 方法，所以可以直接用len获取数据总数
len(ds_demo)

11573

In [6]:
# 用索引直接访问对应的数据，对应 __getitem__ 方法
ds_demo[0]

24000.0

自定义的数据集已经创建好，下面使用官方提供的数据载入器读取数据

## Dataloader

DataLoader提供对Dataset的读取操作，常用参数有：
* **batch_size** (每个batch的大小)
* **shuffle** (是否进行shuffle操作)
* **num_workers** (加载数据时用几个子进程)

下面做一个简单的操作

In [7]:
dl = torch.utils.data.DataLoader(ds_demo,
                                 batch_size=10,
                                 shuffle=True,
                                 num_workers=0)

DataLoader返回一个**可迭代对象**，可以用迭代器分次获取数据

In [8]:
idata = iter(dl)
print(next(idata))

tensor([24000., 24000., 24000., 24000., 24000., 24000., 24000., 24000., 24000.,
        24000.], dtype=torch.float64)


常见用法是用for循环进行遍历

In [9]:
for i, data in enumerate(dl):
    print(i, data)

    # 为了节约空间，这里只循环一遍
    break

0 tensor([24000., 24000., 24000., 24000., 24000., 24000., 24000., 24000., 24000.,
        24000.], dtype=torch.float64)


已经可以通过dataset定义数据集，使用Datalorder载入和遍历数据集，

除了这些，PyTorch还提供torcvision的计算机视觉扩展包，里面封装了

## torchvision 包
torchvision 是PyTorch专门处理图像的库，PyTorch官网的安装教程最后的pip install torchvision 就是安装这个包。

### torchvision.datasets
torchvision.datasets 可以理解为PyTorch团队自定义的dataset，

这些dataset提前处理好了很多的图片数据集，拿来可以直接使用：

- MNIST
- COCO
- Captions
- Detection
- LSUN
- ImageFolder
- Imagenet-12
- CIFAR
- STL10
- SVHN
- PhotoTour

In [10]:
import torchvision.datasets as datasets
trainset = datasets.MNIST(
    # MNIST 数据的加载目录
    root='./data',
    
    # 是否加载数据库的训练集，false加载测试集
    train=True,
    
    # 是否自动下载 MNIST 数据集
    download=True,
    
    # 是否需要对数据进行预处理，none为不进行预处理
    transform=None)

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw




Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw
Processing...
Done!


/Users/jinminyu/opt/anaconda3/lib/python3.8/site-packages/torchvision/datasets/mnist.py:469: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ../torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


### torchvision.models

torchvision不仅提供常用图片数据集，还提供训练好的模型，加载后可以直接使用，或者进行迁移学习
torchvision.models的子模块包含以下模型结构。
- AlexNet
- VGG
- ResNet
- SqueezeNet
- DenseNet

In [12]:
# 可以直接使用训练好的模型，这个与datasets相同，都需要从服务器下载
import socket
socket.gethostbyname("")
import torchvision.models as models
resnet18 = models.resnet18(pretrained=True)

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /Users/jinminyu/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth


### torchvision.transforms
transforms 模块提供一般的图像转换操作类，用作数据处理和数据增强

In [13]:
from torchvision import transforms as transforms

transform = transforms.Compose([
    #四周填充0，把图像随机裁剪成32*32
    transforms.RandomCrop(32, padding=4),  
    
    #图像一半概率翻转，一半概率不翻转
    transforms.RandomHorizontalFlip(), 
    
    #随机旋转
    transforms.RandomRotation((-45, 45)),  
    transforms.ToTensor(),
    
    #R,G,B每层的归一化用到的均值和方差
    transforms.Normalize((0.4914, 0.4822, 0.4465),
                         (0.229, 0.224, 0.225)),  
])

(0.4914, 0.4822, 0.4465), (0.229, 0.224, 0.225)这几个数字是什么意思？

官方的这个帖子有详细的说明:
https://discuss.pytorch.org/t/normalization-in-the-mnist-example/457/21
这是根据ImageNet训练的归一化参数，可以直接使用，认为这个是固定值就可以